# 🔬 Phase 3: Project-First-Bind-Later

## Key Insight from Experts
Current approach does math in dense 384d/768d space, THEN projects.
HDC math works better in high-dimensional space where vectors are quasi-orthogonal.

## Architecture Change
```
OLD: concat(P, H, P-H, P*H) → project → quantize
NEW: P → project → P_hdc
     H → project → H_hdc
     bind(P_hdc, H_hdc) → quantize
```

## Experiments
1. Basic: P_hdc + H_hdc + P_hdc*H_hdc
2. With permutation: roll(P_hdc) * H_hdc (asymmetry)
3. With tanh: non-linearity before binding
4. Two-vector: [P_hdc, H_hdc, diff, prod] → MLP

---

In [ ]:
!pip install -q sentence-transformers datasets
print("✅ Dependencies installed")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import json

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

print(f"PyTorch: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
print(f"\n🔬 Phase 3: Project-First-Bind-Later")

In [ ]:
# Use best encoder from Phase 2
ENCODER_NAME = 'sentence-transformers/nli-mpnet-base-v2'
print(f"Loading {ENCODER_NAME}...")
encoder = SentenceTransformer(ENCODER_NAME)
SEMANTIC_DIM = encoder.get_sentence_embedding_dimension()
print(f"✅ Encoder loaded: {SEMANTIC_DIM}d")

In [ ]:
# Load MNLI
print("\nLoading MNLI...")
dataset = load_dataset("glue", "mnli")

TRAIN_SIZE = 5000
TEST_SIZE = 500

train_data = dataset['train'].shuffle(seed=42).select(range(TRAIN_SIZE))
test_data = dataset['validation_matched'].select(range(TEST_SIZE))

train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

print(f"✅ Train: {TRAIN_SIZE}, Test: {TEST_SIZE}")

In [ ]:
# Pre-compute embeddings
print("\n🔄 Computing embeddings...")

train_p = encoder.encode(list(train_data['premise']), show_progress_bar=True)
train_h = encoder.encode(list(train_data['hypothesis']), show_progress_bar=True)
test_p = encoder.encode(list(test_data['premise']), show_progress_bar=True)
test_h = encoder.encode(list(test_data['hypothesis']), show_progress_bar=True)

print(f"✅ Embeddings: {train_p.shape}")

In [ ]:
# HDC Configuration
HDC_DIM = 4096

# Create projection matrix (same for both P and H)
np.random.seed(42)
projection = np.random.randn(SEMANTIC_DIM, HDC_DIM).astype(np.float32)
projection /= np.linalg.norm(projection, axis=0, keepdims=True)

print(f"✅ Projection matrix: {projection.shape}")

In [ ]:
def ternary_quantize(features, threshold_factor=0.3):
    """Quantize to {-1, 0, +1}."""
    thr = threshold_factor * np.std(features, axis=1, keepdims=True)
    return np.where(features > thr, 1,
                    np.where(features < -thr, -1, 0)).astype(np.float32)

---
## Encoding Strategies: Project-First-Bind-Later
---

In [ ]:
class ProjectFirstStrategies:
    """Different binding strategies after projection to HDC space."""
    
    @staticmethod
    def baseline_concat(p_emb, h_emb, projection):
        """OLD approach: concat first, then project (for comparison)."""
        combined = np.concatenate([p_emb, h_emb, p_emb - h_emb, p_emb * h_emb], axis=1)
        # Need different projection for this
        np.random.seed(42)
        proj_big = np.random.randn(combined.shape[1], HDC_DIM).astype(np.float32)
        proj_big /= np.linalg.norm(proj_big, axis=0, keepdims=True)
        return combined @ proj_big
    
    @staticmethod
    def project_then_bundle(p_emb, h_emb, projection):
        """Project first, then bundle (sum)."""
        p_hdc = p_emb @ projection
        h_hdc = h_emb @ projection
        # Bundle: P + H + (P * H)
        binding = p_hdc * h_hdc
        return p_hdc + h_hdc + binding
    
    @staticmethod
    def project_permute_bind(p_emb, h_emb, projection):
        """Project first, permute P for asymmetry, then bind."""
        p_hdc = p_emb @ projection
        h_hdc = h_emb @ projection
        # Permute P to encode direction (P→H ≠ H→P)
        p_rotated = np.roll(p_hdc, shift=1, axis=1)
        binding = p_rotated * h_hdc
        return p_hdc + h_hdc + binding
    
    @staticmethod
    def project_tanh_bind(p_emb, h_emb, projection):
        """Project, apply tanh, then bind."""
        p_hdc = np.tanh(p_emb @ projection)
        h_hdc = np.tanh(h_emb @ projection)
        binding = p_hdc * h_hdc
        return p_hdc + h_hdc + binding
    
    @staticmethod
    def project_tanh_permute_bind(p_emb, h_emb, projection):
        """Project, tanh, permute, bind — full approach."""
        p_hdc = np.tanh(p_emb @ projection)
        h_hdc = np.tanh(h_emb @ projection)
        p_rotated = np.roll(p_hdc, shift=1, axis=1)
        binding = p_rotated * h_hdc
        return p_hdc + h_hdc + binding
    
    @staticmethod
    def two_vectors_concat(p_emb, h_emb, projection):
        """Project separately, concat with interactions for MLP."""
        p_hdc = p_emb @ projection
        h_hdc = h_emb @ projection
        diff = p_hdc - h_hdc
        prod = p_hdc * h_hdc
        # Concat all: 4 * HDC_DIM = 16384d
        return np.concatenate([p_hdc, h_hdc, diff, prod], axis=1)
    
    @staticmethod
    def two_vectors_tanh(p_emb, h_emb, projection):
        """Same as above but with tanh."""
        p_hdc = np.tanh(p_emb @ projection)
        h_hdc = np.tanh(h_emb @ projection)
        diff = p_hdc - h_hdc
        prod = p_hdc * h_hdc
        return np.concatenate([p_hdc, h_hdc, diff, prod], axis=1)

# Strategies to test
STRATEGIES = {
    '0_baseline_concat': (ProjectFirstStrategies.baseline_concat, HDC_DIM),
    '1_project_bundle': (ProjectFirstStrategies.project_then_bundle, HDC_DIM),
    '2_project_permute': (ProjectFirstStrategies.project_permute_bind, HDC_DIM),
    '3_project_tanh': (ProjectFirstStrategies.project_tanh_bind, HDC_DIM),
    '4_project_tanh_permute': (ProjectFirstStrategies.project_tanh_permute_bind, HDC_DIM),
    '5_two_vectors': (ProjectFirstStrategies.two_vectors_concat, HDC_DIM * 4),
    '6_two_vectors_tanh': (ProjectFirstStrategies.two_vectors_tanh, HDC_DIM * 4),
}

print("📋 Strategies to test:")
for name, (_, dim) in STRATEGIES.items():
    print(f"   {name}: {dim}d")

In [ ]:
# Dataset and Model
class SimpleDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_classes=3, dropout=0.3):
        super().__init__()
        # Adjust hidden dim for larger inputs
        if input_dim > 8000:
            hidden_dim = 1024
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)

def train_and_evaluate(train_features, train_labels, test_features, test_labels,
                       input_dim, num_epochs=25, lr=1e-3):
    """Train MLP and return best accuracy."""
    
    train_dataset = SimpleDataset(train_features, train_labels)
    test_dataset = SimpleDataset(test_features, test_labels)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    model = MLPClassifier(input_dim=input_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    best_acc = 0
    
    for epoch in range(num_epochs):
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            loss = criterion(model(features), labels)
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        model.eval()
        all_preds = []
        with torch.no_grad():
            for features, _ in test_loader:
                features = features.to(device)
                preds = torch.argmax(model(features), dim=1)
                all_preds.extend(preds.cpu().numpy())
        
        acc = accuracy_score(test_labels, all_preds)
        best_acc = max(best_acc, acc)
    
    return best_acc

In [ ]:
print("\n" + "="*60)
print("🔬 TESTING PROJECT-FIRST-BIND-LATER STRATEGIES")
print("="*60)

results = {}

for strategy_name, (strategy_fn, output_dim) in STRATEGIES.items():
    print(f"\n📊 Testing: {strategy_name}")
    
    # Apply strategy
    train_encoded = strategy_fn(train_p, train_h, projection)
    test_encoded = strategy_fn(test_p, test_h, projection)
    
    # Test float version
    print(f"   Float ({output_dim}d)...")
    acc_float = train_and_evaluate(train_encoded, train_labels,
                                    test_encoded, test_labels, output_dim)
    print(f"   Float: {acc_float:.1%}")
    
    # Test ternary version
    print(f"   Ternary...")
    train_ternary = ternary_quantize(train_encoded)
    test_ternary = ternary_quantize(test_encoded)
    acc_ternary = train_and_evaluate(train_ternary, train_labels,
                                      test_ternary, test_labels, output_dim)
    print(f"   Ternary: {acc_ternary:.1%}")
    
    results[strategy_name] = {
        'float': acc_float,
        'ternary': acc_ternary,
        'dim': output_dim,
        'overhead': acc_float - acc_ternary
    }
    
    print(f"   ✅ Overhead: {(acc_float - acc_ternary)*100:.1f}%")

In [ ]:
print("\n" + "="*60)
print("📊 RESULTS SUMMARY")
print("="*60)

# Sort by ternary accuracy
sorted_results = sorted(results.items(), key=lambda x: x[1]['ternary'], reverse=True)

print(f"\n{'Strategy':<30} {'Dim':<8} {'Float':<10} {'Ternary':<10} {'Overhead'}")
print("-" * 70)

baseline_ternary = results['0_baseline_concat']['ternary']

for name, data in sorted_results:
    marker = "🏆" if data['ternary'] == sorted_results[0][1]['ternary'] else "  "
    improvement = (data['ternary'] - baseline_ternary) * 100
    print(f"{marker} {name:<28} {data['dim']:<8} {data['float']:.1%}      {data['ternary']:.1%}      {data['overhead']*100:.1f}%")

In [ ]:
# Find best
best_strategy = sorted_results[0][0]
best_ternary = sorted_results[0][1]['ternary']
best_float = sorted_results[0][1]['float']

# Phase 2 best for comparison
phase2_best = 0.638  # NLI-MPNet full HDC

print("\n" + "="*60)
print("🎯 CONCLUSIONS")
print("="*60)

print(f"\n🏆 Best strategy: {best_strategy}")
print(f"   Float: {best_float:.1%}")
print(f"   Ternary: {best_ternary:.1%}")
print(f"\n📈 vs Phase 2 best ({phase2_best:.1%}): {(best_ternary - phase2_best)*100:+.1f}%")
print(f"📈 vs Baseline concat: {(best_ternary - baseline_ternary)*100:+.1f}%")

if best_ternary > 0.70:
    print(f"\n✅ SIGNIFICANT IMPROVEMENT! Exceeded 70% target.")
    verdict = "TARGET_REACHED"
elif best_ternary > phase2_best + 0.03:
    print(f"\n📈 Good improvement over Phase 2.")
    verdict = "IMPROVEMENT"
else:
    print(f"\n⚠️ Limited improvement. May need different approach.")
    verdict = "LIMITED"

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
ax = axes[0]
names = [n.replace('_', '\n') for n, _ in sorted_results]
float_accs = [d['float'] for _, d in sorted_results]
tern_accs = [d['ternary'] for _, d in sorted_results]

x = np.arange(len(names))
width = 0.35

bars1 = ax.bar(x - width/2, float_accs, width, label='Float', color='lightgreen', edgecolor='black')
bars2 = ax.bar(x + width/2, tern_accs, width, label='Ternary', color='lightcoral', edgecolor='black')

ax.axhline(y=phase2_best, color='blue', linestyle='--', label=f'Phase 2 best ({phase2_best:.1%})')
ax.axhline(y=0.33, color='gray', linestyle=':', alpha=0.5)
ax.set_ylabel('Accuracy')
ax.set_title('Project-First-Bind-Later Strategies')
ax.set_xticks(x)
ax.set_xticklabels(names, fontsize=8)
ax.legend(loc='lower right')
ax.set_ylim(0.3, max(float_accs) + 0.05)

# Improvement chart
ax = axes[1]
improvements = [(d['ternary'] - baseline_ternary) * 100 for _, d in sorted_results]
colors = ['lightgreen' if i > 0 else 'lightcoral' for i in improvements]
bars = ax.bar(names, improvements, color=colors, edgecolor='black')
ax.axhline(y=0, color='black', linewidth=0.5)
ax.axhline(y=(phase2_best - baseline_ternary) * 100, color='blue', linestyle='--', 
           label=f'Phase 2 improvement')
ax.set_ylabel('Improvement vs Baseline (%)')
ax.set_title('Improvement over Baseline Concat')
ax.set_xticklabels(names, fontsize=8, rotation=45, ha='right')

for bar, imp in zip(bars, improvements):
    y_pos = bar.get_height() + 0.5 if imp >= 0 else bar.get_height() - 1
    ax.text(bar.get_x() + bar.get_width()/2, y_pos,
            f'{imp:+.1f}%', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('phase3_project_first_results.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save results
output = {
    'experiment': 'Phase 3: Project-First-Bind-Later',
    'dataset': 'MNLI',
    'encoder': ENCODER_NAME,
    'train_size': TRAIN_SIZE,
    'test_size': TEST_SIZE,
    'hdc_dim': HDC_DIM,
    'results': {k: {kk: float(vv) if isinstance(vv, (np.floating, float)) else vv 
                    for kk, vv in v.items()} 
                for k, v in results.items()},
    'best_strategy': best_strategy,
    'best_float': float(best_float),
    'best_ternary': float(best_ternary),
    'phase2_best': phase2_best,
    'improvement_vs_phase2': float(best_ternary - phase2_best),
    'verdict': verdict,
    'timestamp': datetime.now().isoformat()
}

with open('phase3_project_first_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("\n✅ Results saved!")
print(json.dumps(output, indent=2))

In [ ]:
print("\n" + "="*60)
print("📋 NEXT STEPS")
print("="*60)

print(f"""
Current best: {best_ternary:.1%} ({best_strategy})

Options to try next:
1. Role-Filler Binding (VSA-style)
2. Increase HDC dimension (8192d)
3. Learned projection matrix
4. Circular convolution instead of element-wise mult
5. Combine best strategies
""")

In [ ]:
from google.colab import files
files.download('phase3_project_first_results.json')
files.download('phase3_project_first_results.png')